In [2]:
import pyboy
from pyboy import PyBoy, WindowEvent
from pyboy.plugins import game_wrapper_super_mario_land as mario_wrapper
import gym
import numpy as np
import keras
import rl
from rl import processors
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout, InputLayer, Flatten
from keras.optimizers import Adam

from collections import deque


# Env

In [3]:
class marioEnvironment(gym.Env):
    
    def __init__(self):
        self.pyboy = PyBoy('/home/dani/Downloads/Super Mario Land (World).gb',window_scale =10 ,game_wrapper=True)
        self.mario_wrapper = self.pyboy.game_wrapper()
        self.steps = 0
        self.mario_wrapper.start_game()
        self.mario_wrapper.post_tick()
        self.reward = self.mario_wrapper.fitness
        self.done = False
        
#         self.dicAcciones_pad = {0: WindowEvent.PRESS_ARROW_UP, 1: WindowEvent.PRESS_ARROW_RIGHT, 2: WindowEvent.PRESS_ARROW_LEFT,
#                                }
#         self.dicAcciones_pad_fin = {0: WindowEvent.RELEASE_ARROW_UP, 1: WindowEvent.RELEASE_ARROW_RIGHT, 2: WindowEvent.RELEASE_ARROW_LEFT}
#         self.dicAcciones_A = {0: WindowEvent.PRESS_ARROW_UP, 1: WindowEvent.PRESS_BUTTON_A}
#         self.dicAcciones_B = {0: WindowEvent.PRESS_ARROW_UP, 1: WindowEvent.PRESS_BUTTON_B}

#         self.action_space = gym.spaces.Tuple((gym.spaces.MultiDiscrete([3,2,2]), gym.spaces.Box(1, 10,shape=(1,) ,dtype=np.int32)))# numero de acciones disponibles 5: (NOOP,UP,DOWN,LEFT,RIGHT), 2:(NOOP, pulsar A o B)
#         self.action_space = gym.spaces.MultiDiscrete([3,2,2])
        self.action_space = gym.spaces.Discrete(10)
#         self.observation_space = gym.spaces.Dict({'pantalla': gym.spaces.Box(low = 0,high= 500, shape=(16, 20), dtype = np.uint32),
#                                                   'time': gym.spaces.Box(low=0, high=400, shape=(1,), dtype=int), 'level_progress': gym.spaces.Box(0,1000, shape=(1,), dtype=int)})
        self.observation_space = gym.spaces.Box(low = 0,high= 500, shape=(16, 20), dtype = np.uint32)

        
    def _next_observation(self):
            #asdasd
#         obs = {'pantalla': np.array(self.mario_wrapper.game_area()),
#               'time': np.array([self.mario_wrapper.time_left]),
#               'level': np.array([self.mario_wrapper.level_progress])}
        obs = np.array(self.mario_wrapper.game_area()).reshape(16,20)
        return obs
    
    def move_mario(self, action):
        if action ==0:
            for i in range(10):
                self.pyboy.tick()
            self.mario_wrapper.post_tick()
            
        elif action == 1:
            self.pyboy.send_input(WindowEvent.PRESS_ARROW_RIGHT)
            for i in range(10):
                self.pyboy.tick()
            self.pyboy.send_input(WindowEvent.RELEASE_ARROW_RIGHT)
            self.mario_wrapper.post_tick()
            
        elif action == 2:
            self.pyboy.send_input(WindowEvent.PRESS_ARROW_LEFT)
            for i in range(10):
                self.pyboy.tick()
            self.pyboy.send_input(WindowEvent.RELEASE_ARROW_LEFT)
            self.mario_wrapper.post_tick()
            
        elif action == 3:
            self.pyboy.send_input(WindowEvent.PRESS_BUTTON_A)
            for i in range(55):
                self.pyboy.tick()
            self.pyboy.send_input(WindowEvent.RELEASE_BUTTON_A)
            self.mario_wrapper.post_tick()
            
        elif action == 4:
            self.pyboy.send_input(WindowEvent.PRESS_BUTTON_A)
            self.pyboy.send_input(WindowEvent.PRESS_ARROW_RIGHT)
            for i in range(55):
                self.pyboy.tick()
            self.pyboy.send_input(WindowEvent.RELEASE_BUTTON_A)
            self.pyboy.send_input(WindowEvent.RELEASE_ARROW_RIGHT)
            self.mario_wrapper.post_tick()
            
        elif action == 5:
            self.pyboy.send_input(WindowEvent.PRESS_BUTTON_A)
            self.pyboy.send_input(WindowEvent.PRESS_ARROW_LEFT)
            for i in range(55):
                self.pyboy.tick()
            self.pyboy.send_input(WindowEvent.RELEASE_BUTTON_A)
            self.pyboy.send_input(WindowEvent.RELEASE_ARROW_LEFT)
            self.mario_wrapper.post_tick()
            
        elif action == 6:
            self.pyboy.send_input(WindowEvent.PRESS_BUTTON_B)
            self.pyboy.send_input(WindowEvent.PRESS_ARROW_RIGHT)
            for i in range(10):
                self.pyboy.tick()
            self.pyboy.send_input(WindowEvent.RELEASE_BUTTON_B)
            self.pyboy.send_input(WindowEvent.RELEASE_ARROW_RIGHT)
            self.mario_wrapper.post_tick()
        elif action == 7:
            self.pyboy.send_input(WindowEvent.PRESS_BUTTON_B)
            self.pyboy.send_input(WindowEvent.PRESS_ARROW_LEFT)
            for i in range(10):
                self.pyboy.tick()
            self.pyboy.send_input(WindowEvent.RELEASE_BUTTON_B)
            self.pyboy.send_input(WindowEvent.RELEASE_ARROW_LEFT)
            self.mario_wrapper.post_tick()
            
        elif action == 8:
            self.pyboy.send_input(WindowEvent.PRESS_BUTTON_A)
            self.pyboy.send_input(WindowEvent.PRESS_BUTTON_B)
            self.pyboy.send_input(WindowEvent.PRESS_ARROW_RIGHT)
            for i in range(55):
                self.pyboy.tick()
            self.pyboy.send_input(WindowEvent.RELEASE_BUTTON_A)
            self.pyboy.send_input(WindowEvent.RELEASE_BUTTON_B)
            self.pyboy.send_input(WindowEvent.RELEASE_ARROW_RIGHT)
            self.mario_wrapper.post_tick()
            
        elif action == 9:
            self.pyboy.send_input(WindowEvent.PRESS_BUTTON_A)
            self.pyboy.send_input(WindowEvent.PRESS_BUTTON_B)
            self.pyboy.send_input(WindowEvent.PRESS_ARROW_LEFT)
            for i in range(55):
                self.pyboy.tick()
            self.pyboy.send_input(WindowEvent.RELEASE_BUTTON_A)
            self.pyboy.send_input(WindowEvent.RELEASE_BUTTON_B)
            self.pyboy.send_input(WindowEvent.RELEASE_ARROW_LEFT)
            self.mario_wrapper.post_tick()
            
        
            
        
        
    def reset(self):
        self.mario_wrapper.reset_game()
        self.obs = self._next_observation()
        return self.obs
    
    
    def step(self, action):
        
#         n_frames = action[1][0]
#         print('action: {}'.format(action))
        lives = self.mario_wrapper.lives_left
        self.move_mario(action)
        if lives > self.mario_wrapper.level_progress:
            print('\nNOW')
        obs = self._next_observation()
        reward = self.mario_wrapper.fitness - self.reward
        done = self.mario_wrapper.level_progress > 1000
        if done:
            reward = 100000
        
        return obs, reward, done, {}
        
        
    def _take_action(self, action):
        pass
        
    def render(self):
        pass
    
    def close(self):
        self.pyboy.stop()
        
    
        

# SARSA

In [4]:
from rl.agents import SARSAAgent
from rl.policy import EpsGreedyQPolicy

policy = EpsGreedyQPolicy()
env = marioEnvironment()

In [5]:
def agent(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape = states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model
  
model = agent((1,16,20), env.action_space.n)

In [6]:
sarsa = SARSAAgent(model = model, policy = policy, nb_actions = env.action_space.n)
sarsa.compile('adam', metrics = ['mse'])


In [7]:
sarsa.fit(env, nb_steps = 50000, visualize = False, verbose = 1)

Training for 50000 steps ...
Interval 1 (0 steps performed)

  146/10000 [..............................] - ETA: 1:55:19 - reward: 32610.9589 
NOW
  147/10000 [..............................] - ETA: 1:55:31 - reward: 76867.8912
NOW
  148/10000 [..............................] - ETA: 1:55:42 - reward: 120526.7568
NOW
  149/10000 [..............................] - ETA: 1:55:53 - reward: 163599.5973
NOW
  150/10000 [..............................] - ETA: 1:56:04 - reward: 206098.1333
NOW
  151/10000 [..............................] - ETA: 1:56:15 - reward: 248033.7748
NOW
  152/10000 [..............................] - ETA: 1:56:26 - reward: 289417.6316
NOW
  153/10000 [..............................] - ETA: 1:56:36 - reward: 330260.5229
NOW
  154/10000 [..............................] - ETA: 1:56:47 - reward: 370572.9870
NOW
  155/10000 [..............................] - ETA: 1:56:57 - reward: 410365.2903
NOW
  156/10000 [..............................] - ETA: 1:57:07 - reward: 449647.435